In [ ]:
%pip install pandas langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt langchain_upstage

In [1]:
import os 
import dotenv 

dotenv.load_dotenv()

True

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from pprint import pprint

from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# GPT_API_KEY = os.environ.get("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [3]:
folder_path = '../data'

In [2]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

NameError: name 'folder_path' is not defined

In [2]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [3]:
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-index'

In [ ]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [4]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [11]:
query = ('결제 후 주문 취소가 가능한가요?')

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

* [SIM=0.498854] : 8
Question: Q[결제수단] 이미 주문했는데 결제 수단을 변경할 수 있나요?
Answer: 아니요. 상품을 주문/결제한 후에는 결제 수단 변경이 불가합니다. 상품이 아직 배송 전이라면 주문목록에서 주문을 취소한 후 원하는 결제 수단으로 다시 주문/결제해 주시기 바랍니다. 상품이 이미 배송을 시작했다면 배송 완료 후 반품을 접수해 주시기 바랍니다. 단 상품 배송 이후 단순 변심으로 반품할 경우 왕복 배송비가 부과될 수 있습니다.
 주문 취소하기
마이쿠팡 → 주문목록 → 상품선택 → [주문취소] 
이후 각 단계에 해당하는 항목을 선택하여 취소를 완료합니다.
 쿠페이 결제수단 관리하기
마이쿠팡 → 결제수단∙쿠페이 → [결제수단 관리] 
이후 원하는 결제수단을 등록 및 삭제합니다.
 단순 변심 반품 비용 (클릭)
keywords: ORDER
count: 0 [{'row': 8.0, 'source': '../data/ORDER_main_qna_list.csv'}]
* [SIM=0.498439] : 9
Question: Q[취소 철회] 주문취소 신청을 철회(취소)하고 싶어요.
Answer: 배송 상품은 주문을 취소하면 바로 결제 취소가 적용되어 이후 주문취소를 철회(취소) 할 수 없습니다. 이미 주문취소를 하셨다면 재구매를 부탁드립니다.
keywords: CANCEL
count: 0 [{'row': 9.0, 'source': '../data/CANCEL_main_qna_list.csv'}]
* [SIM=0.484393] : 44
Question: Q[취소] 신용카드로 결제한 주문을 취소했는데 카드대금이 청구되었습니다.
Answer: 신용카드로 결제한 주문을 취소하면 신용카드 승인 취소도 자동으로 진행됩니다. 그런데 만약 취소한 주문에 대해 카드대금 청구를 받으셨다면 이는 주문이 취소되기 전에 카드사에서 이미 청구서를 작성했기 때문입니다. 이 경우 청구된 카드대금이 취소되기까지는 특정 시간이 소요되며 이는 각 카드사마다 다릅니다. 

# LLM 질의 테스트

In [ ]:
%pip install langchain_google_genai

In [5]:
retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.3보다 낮게 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

    

In [8]:
# 챗봇에 질의
query = '결제 후 주문 취소가 가능한가요?'

answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')

Answer : 네, 결제 후 주문 취소가 가능합니다.
 하지만 상품 배송 여부에 따라 가능 여부와 절차가 다르니, 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.
 상품 배송 전이라면 주문 취소가 가능하며, 배송 중이거나 완료된 경우에는 반품 절차를 안내해 드리겠습니다.



**각 조건별 대응 방법**

* **상품 배송 전:** "주문 번호를 확인했습니다.
 현재 상품 배송 전이므로 주문 취소가 가능합니다.
 주문 취소를 도와드리겠습니다.
 잠시만 기다려주세요.
" (주문 취소 진행 후) "주문이 취소되었습니다.
 불편을 드려 죄송합니다.
"

* **상품 배송 중 또는 완료:** "주문 번호를 확인했습니다.
 현재 상품이 배송 중이거나 배송 완료된 상태라 주문 취소는 어렵습니다.
 단순 변심으로 인한 반품을 원하시면, 반품 절차를 안내해 드리겠습니다.
 번거로우시겠지만, 반품 신청을 위해 필요한 정보를 안내해 드릴 테니, 잠시만 기다려주세요.
" (반품 절차 안내 후) "반품 절차에 대한 안내를 드렸습니다.
 궁금한 점이 있으시면 언제든지 문의해주세요.
"

* **주문 번호 확인 불가:** "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.
"


.


#### Ollama 버전 

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
llm = ChatOllama(
    model="llama3.1:8b",  # 사용할 모델을 지정합니다.
)

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

prompt = ChatPromptTemplate.from_template(template)

# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)
# 유사도 보기
# results = database.similarity_search_with_score(query=query, k=3)
# pprint(results)
query = '주문 후 배송 주소를 변경할 수 있나요?'
results = database.similarity_search_with_score(query=query, k=3)
pprint(results)

print()
answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:  
    print(ans + '.')


[(Document(id='31ee5dfd-3b6f-4255-a62c-ea7b482ac8d0', metadata={'row': 48.0, 'source': '../data/DELIVERY_main_qna_list.csv'}, page_content=': 48\nQuestion: Q[배송지] 주문 후 결제까지 완료했는데 배송지를 변경하고 싶어요.\nAnswer: 주문/결제 직후라면 아래의 경로에서 배송지 변경이 가능합니다. 단 상품 종류 및 준비 상태에 따라 배송지 변경이 불가할 수 있습니다.\n 주문상품 배송지 변경하기\n① 마이쿠팡 → 주문목록 → [주문상세보기] 선택\n② ‘받는사람 정보’의 ‘받는주소’에서 [변경하기] 선택\n※ 위 [변경하기] 버튼이 없으면 배송지를 변경할 수 있는 시간이 경과한 것입니다.\nkeywords: DELIVERY\ncount: 0'),
  0.536497891),
 (Document(id='dc6869eb-e9e7-43e7-bcb4-0805dc43d612', metadata={'row': 38.0, 'source': '../data/DELIVERY_main_qna_list.csv'}, page_content=': 38\nQuestion: Q[배송지] 배송 중에 배송지 및 배송요청사항을 변경할 수 있나요?\nAnswer: 아니요. 상품이 이미 출고되어 배송 중인 상태에서는 배송지 및 배송요청사항을 변경할 수 없습니다.\nkeywords: DELIVERY\ncount: 0'),
  0.49331823),
 (Document(id='709c58dd-c3aa-4017-bc0c-90c1b0d82032', metadata={'row': 49.0, 'source': '../data/DELIVERY_main_qna_list.csv'}, page_content=': 49\nQuestion: Q[로켓배송] 새벽배송 주문 후 배송지를 변경할 수 있나요?\nAnswer: 상품에 종류에 따라 다릅니다. 새벽배송 전용인 로켓프레시 상품은 주문/결제 

# Cache DB 연동

In [6]:
import pymysql as sql
from pprint import pprint
import hashlib

In [7]:
# db 불러오기
conn = sql.connect(host= "localhost", user="root", password='1234', db='cache_db', charset='utf8')
table_name = "cache"
require_min_length = 5

In [8]:
def select_all_data():
    cursor = conn.cursor()
    cursor.execute(f"select * from {table_name};")

    cur_db = cursor.fetchall()
    for d in cur_db:
        pprint(d)

In [9]:
def validation_data(query):

    return len(query) > require_min_length

# 데이터 삽입
def insert_data(query):
    if not validation_data(query):
        print(f"최소 {require_min_length}자 이상의 질문을 입력해 주세요")
        return

    try:
        with conn.cursor() as cur:
            query = "insert into cache (created_at, query, hashed_docs, created_answer) values (%s, %s, %s, %s);"
            cur.execute("use cache")

            cur.execute(query, (
                
            ))

            conn.commit()

    finally:
        conn.close()

In [26]:
# DB에 현재 md5 정보와 일치하는 문서가 있는지 확인
def is_exist_doc(query_md5):
    try:
        with conn.cursor() as cur:
            find_query_md5 = f"select * from {table_name} where hashed_docs = '{query_md5}';"
            cur.execute(find_query_md5)
            
            exist_doc = cur.fetchall()
            return exist_doc
            
    except Exception as e:
        print(e)
        

def query_to_md5(query):
    return ",".join(sorted([hashlib.md5(result.page_content.encode()).hexdigest() for result in database.similarity_search(query=query, k=3)]))

In [33]:
def get_exist_llm_answer(query):
    query_md5 = query_to_md5(query)
    docs = is_exist_doc(query_md5)
    
    # 일치하는 docs가 없다면
    if len(docs) == 0:
        # generate_and_store_llm()
        
        return "Answer: ** 해당하는 질문이 Cache DB상에 존재하지 않습니다.\n\n"
    
    else:
        answer = docs[0][4]
        return answer
        

In [11]:
test_md5 = "5aec9b0cbb8e768576a0bc99157cbb8a,72f328e328ed93624991283a5e2e3c09,de14d9dd57ef5fcc9a3f113baea01aa7"

In [12]:
doc = is_exist_doc(test_md5)

select * from cache where hashed_docs = '5aec9b0cbb8e768576a0bc99157cbb8a,72f328e328ed93624991283a5e2e3c09,de14d9dd57ef5fcc9a3f113baea01aa7';


In [34]:
answer = get_exist_llm_answer(query)

In [36]:
temp = answer.split('Answer:')

In [ ]:
temp